# RDL analytics interface

In [13]:
import ipywidgets as widgets
from IPython.display import display, Image
from PIL import Image as PILImage
from io import BytesIO
import subprocess

# Load the image for the top-right corner
with open("rdl_logo.png", "rb") as img_file:
    img = PILImage.open(img_file)
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    img_data = buffer.getvalue()

# Create UI elements with adjusted label widths
country_selector = widgets.Text(value='TUN', description='Country:', layout=widgets.Layout(width='400px'), style={'description_width': '150px'})
hazard_selector = widgets.Dropdown(options=[
    ('Fluvial Undefended', 'FLUVIAL_UNDEFENDED'), 
    ('Fluvial Defended', 'FLUVIAL_DEFENDED'),
    ('Pluvial Defended', 'PLUVIAL_DEFENDED'), 
    ('Coastal Undefended', 'COASTAL_UNDEFENDED'), 
    ('Coastal Defended', 'COASTAL_DEFENDED')
], value='FLUVIAL_UNDEFENDED', description='Hazard type:', layout=widgets.Layout(width='400px'), style={'description_width': '150px'})

hazard_threshold_slider = widgets.IntSlider(value=20, min=0, description='Hazard threshold:', layout=widgets.Layout(width='420px'), style={'description_width': '150px'})

period_selector = widgets.Dropdown(options=['2020', '2030', '2050', '2080'], value='2020', description='Hazard period:', layout=widgets.Layout(width='400px'), style={'description_width': '150px'})
scenario_selector = widgets.Dropdown(options=[
    ('SSP1-2.6', 'SSP1_2.6'),
    ('SSP2-4.5', 'SSP2_4.5'),
    ('SSP3-7.0', 'SSP3_7.0'),
    ('SSP5-8.5', 'SSP5_8.5')
], value='SSP3_7.0', description='Climate scenario:', layout=widgets.Layout(width='400px'), style={'description_width': '150px'})

exposure_selector = widgets.Dropdown(options=[
    ('Population', 'POP'),
    ('Built-up', 'BU'),
    ('Agriculture', 'AGR')
], value='POP', description='Exposure:', layout=widgets.Layout(width='400px'), style={'description_width': '150px'})

custom_exposure_radio = widgets.RadioButtons(options=['Default exposure', 'Custom exposure'], description='', layout=widgets.Layout(width='400px'))
custom_exposure_textbox = widgets.Text(value='', placeholder='Enter filename (without extension)', layout=widgets.Layout(width='400px'))

approach_selector = widgets.Dropdown(options=[('Classes', 'Classes'), ('Function', 'Function')], value='Classes', description='Approach:', layout=widgets.Layout(width='400px'), style={'description_width': '150px'})

class_edges_table = widgets.VBox()

# Define add_class_button before using it
add_class_button = widgets.Button(description="Add Class", layout=widgets.Layout(width='150px'))

approach_selector.observe(update_class_edges_table, 'value')
update_class_edges_table()

adm_level_selector = widgets.Dropdown(options=[(str(i), i) for i in range(1, 4)], value=1, description='Adm level:', layout=widgets.Layout(width='400px'), style={'description_width': '150px'})

# Function to update scenario dropdown visibility
def update_scenario_visibility(*args):
    scenario_selector.layout.display = 'none' if period_selector.value == '2020' else 'block'

period_selector.observe(update_scenario_visibility, 'value')
update_scenario_visibility()

# Function to update custom exposure text visibility
def update_custom_exposure_visibility(*args):
    custom_exposure_textbox.layout.display = 'none' if custom_exposure_radio.value == 'Default exposure' else 'block'

custom_exposure_radio.observe(update_custom_exposure_visibility, 'value')
update_custom_exposure_visibility()

# Functions to dynamically add class edges

def create_class_row(index):
    delete_button = widgets.Button(description="Delete", layout=widgets.Layout(width='70px'))
    row = widgets.HBox([
        widgets.Label(f'Class {index}:', layout=widgets.Layout(width='100px')),
        widgets.FloatText(value=0.0, description='', layout=widgets.Layout(width='100px')),
        delete_button
    ])
    delete_button.on_click(lambda b: delete_class(row))
    return row

def delete_class(row):
    class_edges_table.children = [child for child in class_edges_table.children if child != row]
    # Renumber the remaining classes
    for i, child in enumerate(class_edges_table.children[:-1]):  # Exclude the "Add Class" button
        child.children[0].value = f'Class {i+1}:'

def add_class(button):
    new_index = len(class_edges_table.children)
    class_edges_table.children = list(class_edges_table.children[:-1]) + [create_class_row(new_index)] + [add_class_button]

add_class_button.on_click(add_class)

def update_class_edges_table(*args):
    if approach_selector.value == 'Classes':
        # Clear existing table
        class_edges_table.children = []
        # Add initial class rows
        class_edges_table.children = [create_class_row(i) for i in range(1, 4)]
        # Add the "Add Class" button
        class_edges_table.children += (add_class_button,)
    else:
        class_edges_table.children = []

    # Ensure at least one class remains
    if len(class_edges_table.children) == 1:  # Only "Add Class" button remains
        class_edges_table.children = [create_class_row(1)] + list(class_edges_table.children)

approach_selector.observe(update_class_edges_table, 'value')
update_class_edges_table()

# Button to run the script
run_button = widgets.Button(description="Run Analysis", layout=widgets.Layout(width='400px'), button_style='danger')
output = widgets.Output()

def run_analysis_script(b):
    with output:
        output.clear_output()
        # Gather input values
        country = country_selector.value
        haz_cat = hazard_selector.value
        min_haz_slider = hazard_threshold_textbox.value
        period = period_selector.value
        scenario = scenario_selector.value if period != '2020' else ''
        exp_cat_list = [exposure_selector.value]
        exp_nam_list = [custom_exposure_textbox.value] if custom_exposure_radio.value == 'Custom exposure' else [exposure_selector.value]
        analysis_app = approach_selector.value
        adm_level = adm_level_selector.value

        # Dummy call to the script (replace with actual subprocess call in your environment)
        command = ["python3", "try.py"]
        print(f"Running: {command}")
        # Uncomment the line below in your environment
        # subprocess.run(command, check=True)

run_button.on_click(run_analysis_script)

# Displaying the UI
header_html = f"""
<div style='
    background: linear-gradient(to bottom, #003366, transparent);
    padding: 10px;
    border-radius: 10px 10px 0 0;
    display: flex;
    justify-content: space-between;
    align-items: flex-start;
'>
    <div align="center">
        <h1 style='color: #FFFFFF; margin: 0;'>RISK DATA LIBRARY</h1>
        <h2 style='color: #e19f0b; margin: 0;'><b>ANALYTICAL TOOL</b></h2>
    </div>
    <img src="rdl_logo.png" style="width: 150px; height: auto;">
</div>
"""

header = widgets.HTML(value=header_html, layout=widgets.Layout(width='100%'))

# Layout the widgets in a form
form_items = [
    header,
    country_selector,
    hazard_selector,
    hazard_threshold_slider,
    period_selector,
    scenario_selector,
    exposure_selector,
    custom_exposure_radio,
    custom_exposure_textbox,
    approach_selector,
    class_edges_table,
    adm_level_selector,
    run_button,
    output,
]

form = widgets.VBox(form_items, layout=widgets.Layout(
    background='#003366', 
    padding='0',  # Remove padding here
    border_radius='10px', 
    width='600px'
))

form.children[0].layout.padding = '10px 10px 10px 0'
display(form)
